# Efficientnet b3
* Efficientnet b3 적용
    - Efficientnet b3
    - dropout, normalization 적용
    - GELU
* Augmentation
    - Transpose # 행렬 스왑
    - HorizontalFlip # 좌우 반전
    - VerticalFlip # 상하 반전
    - CoarseDrop
* lr scheduler
    - Custom CosineAnnealingWarmUpRestarts
    - T_hold : min_lr * 10 적용
* optimizer
    - AdamW
* focal loss 적용
* cutmix
    - lam `0~0.25`, `0.75~1`
* k-fold
* pacience 
    - 10 -> 12
* result
    - fold0 : Epoch [0-47], lr : [0.000010], Train Loss : [0.4373], Val Loss : [0.3877], Val F1 Score : [0.8103]
    - fold1 : Epoch [1-43], lr : [0.000058], Train Loss : [0.3590], Val Loss : [0.3707], Val F1 Score : [0.8233]
    - fold2 : Epoch [2-42], lr : [0.000006], Train Loss : [0.4004], Val Loss : [0.4116], Val F1 Score : [0.8271]
    - fold3 : Epoch [3-18], lr : [0.000006], Train Loss : [0.5786], Val Loss : [0.4356], Val F1 Score : [0.7953]
    - fold4 : Epoch [4-34], lr : [0.000020], Train Loss : [0.4358], Val Loss : [0.4252], Val F1 Score : [0.8072]
        - 40587
        - public 점수 : 0.8351786096
        - private 점수 : 0.8365907466


In [1]:
import gc
import os
import random
import warnings
from datetime import datetime

import albumentations as A
import cv2
import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from transformers import EfficientNetModel

warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'IMG_SIZE': 300,
    'EPOCHS': 1000,
    'LEARNING_RATE': 6e-6,
    'BATCH_SIZE': 16,
    'PATIENCE': 12,
    'WARMUP': 6,
    'K-FOLD': 5,
    'FILENAME': 'kfold_efficientnet',
    'SEED': 6
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../../data/'

In [6]:
df = pd.read_csv(os.path.join(data_path, 'train.csv'))
df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'artist'] = 'Alfred Sisley'
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [7]:
artists = df.groupby('artist')[['id']].count().rename(columns={'id':'count'}).reset_index()

In [8]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [9]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values
    return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values, df['artist'].values

In [10]:
from torchvision.transforms import ToTensor


class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms if transforms else ToTensor()

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']

        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_paths)

In [11]:
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    # A.ShiftScaleRotate(p=0.5), # 랜덤하게 옮기고, scale, 회전
    # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5), # 빛깔, 색조, 값 변환
    # A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5), # 명도 대비
    # A.ChannelShuffle(), # RGB 채널 간 shuffle
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

validation_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [12]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [13]:
class EfficientNetModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNetModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b3', pretrained=True, num_classes=512)
        self.classifier = nn.Sequential(
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [14]:
from torchsummary import summary

model = EfficientNetModel()
summary(model, (3,300,300))

Layer (type:depth-idx)                             Output Shape              Param #
├─EfficientNet: 1-1                                [-1, 512]                 --
|    └─Conv2d: 2-1                                 [-1, 40, 150, 150]        1,080
|    └─BatchNormAct2d: 2-2                         [-1, 40, 150, 150]        --
|    |    └─Identity: 3-1                          [-1, 40, 150, 150]        --
|    |    └─SiLU: 3-2                              [-1, 40, 150, 150]        --
|    └─Sequential: 2-3                             [-1, 384, 10, 10]         --
|    |    └─Sequential: 3-3                        [-1, 24, 150, 150]        3,504
|    |    └─Sequential: 3-4                        [-1, 32, 75, 75]          48,118
|    |    └─Sequential: 3-5                        [-1, 48, 38, 38]          110,912
|    |    └─Sequential: 3-6                        [-1, 96, 19, 19]          638,700
|    |    └─Sequential: 3-7                        [-1, 136, 19, 19]         1,387,760
|    |  

Layer (type:depth-idx)                             Output Shape              Param #
├─EfficientNet: 1-1                                [-1, 512]                 --
|    └─Conv2d: 2-1                                 [-1, 40, 150, 150]        1,080
|    └─BatchNormAct2d: 2-2                         [-1, 40, 150, 150]        --
|    |    └─Identity: 3-1                          [-1, 40, 150, 150]        --
|    |    └─SiLU: 3-2                              [-1, 40, 150, 150]        --
|    └─Sequential: 2-3                             [-1, 384, 10, 10]         --
|    |    └─Sequential: 3-3                        [-1, 24, 150, 150]        3,504
|    |    └─Sequential: 3-4                        [-1, 32, 75, 75]          48,118
|    |    └─Sequential: 3-5                        [-1, 48, 38, 38]          110,912
|    |    └─Sequential: 3-6                        [-1, 96, 19, 19]          638,700
|    |    └─Sequential: 3-7                        [-1, 136, 19, 19]         1,387,760
|    |  

In [15]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [16]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

        return self.early_stop

In [17]:
def rand_bbox(size, lam):   # size : [B, C, W, H]
    W = size[2] # 이미지의 width
    H = size[3] # 이미지의 height
    cut_rat = np.sqrt(1. - lam)  # 패치 크기의 비율 정하기
    cut_w = np.int32(W * cut_rat)  # 패치의 너비
    cut_h = np.int32(H * cut_rat)  # 패치의 높이

    # uniform
    # 기존 이미지의 크기에서 랜덤하게 값을 가져옵니다.(중간 좌표 추출)
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # 패치 부분에 대한 좌표값을 추출합니다.
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [18]:
import math
from torch.optim.lr_scheduler import LRScheduler

class CosineAnnealingWarmUpRestarts(LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1, T_hold=5e-5):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        self.T_hold = T_hold # custom
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        if self.eta_max < self.T_hold:
            self.eta_max = self.T_hold
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [19]:
time_now = datetime.now()
run_id = time_now.strftime("%Y%m%d%H%M%S")
os.makedirs(os.path.join(data_path, f'./runs/{run_id}'), exist_ok=True)
print(f'{run_id=}')

run_id='20231227082319'


In [20]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [21]:
def train(epoch, model, optimizer, criterion, train_loader, device, beta=1, cutmix_prob=0.5, lr_scheduler=None):
    model.train()
    train_loss = []
    lr_list = []

    for idx, (img, label) in enumerate(train_loader):
        img, label = img.float().to(device), label.long().to(device)

        optimizer.zero_grad()
        r = np.random.rand(1)
        if beta > 0 and r < cutmix_prob:
            lam = np.random.beta(beta, beta)
            if lam < 0.5:
                lam = lam/2 # 0~0.25
            else:
                lam = 0.5 + lam/2 # 0.75~1
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = label
            target_b = label[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]

            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))

            model_pred = model(img)
            loss = criterion(model_pred, target_a) * lam + criterion(model_pred, target_b) * (1. - lam)
        else:
            model_pred = model(img)
            loss = criterion(model_pred, label)

        loss.backward()
        if ((epoch-1)*len(train_loader) + idx)%10 == 0:
            lr_list.append(optimizer.param_groups[0]['lr'])
        optimizer.step()
        train_loss.append(loss.item())
        if lr_scheduler:
            lr_scheduler.step()
    return np.mean(train_loss), lr_list

In [22]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    model_preds = []
    true_labels = []

    val_loss = []

    with torch.no_grad():
        for img, label in iter(test_loader):
            img, label = img.float().to(device), label.long().to(device)

            model_pred = model(img)

            loss = criterion(model_pred, label)

            val_loss.append(loss.item())

            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()

    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [23]:
def train_epoch(k, model, optimizer, train_loader, test_loader, lr_scheduler, device):
    model.to(device)

    criterion = FocalLoss(alpha=1, gamma=2).to(device)
    early_stopping = EarlyStopping(patience=CFG['PATIENCE'], verbose=True)

    best_score = 0
    lr_list = []
    train_loss_list, val_loss_list = [], []

    for epoch in range(1,CFG["EPOCHS"]+1):
        tr_loss, lr_ = train(epoch, model, optimizer, criterion, train_loader, device, beta=1, lr_scheduler=lr_scheduler)
        val_loss, val_score = validation(model, criterion, test_loader, device)
        train_loss_list.append(tr_loss)
        val_loss_list.append(val_loss)

        if lr_scheduler is not None:
            lr_list.extend(lr_)

        if best_score < val_score:
            print(f'**Epoch [{k}-{epoch}], lr : [{lr_[-1]:.6f}], Train Loss : [{tr_loss:.4f}], Val Loss : [{val_loss:.4f}], Val F1 Score : [{val_score:.4f}]')
            best_score = val_score
            torch.save(model, os.path.join(data_path, f'runs/{run_id}/best_model_{k}.pt'))
        else:
            print(f'Epoch [{k}-{epoch}], lr : [{lr_[-1]:.6f}], Train Loss : [{tr_loss:.4f}], Val Loss : [{val_loss:.4f}], Val F1 Score : [{val_score:.4f}]')
        clear_mem()
        if early_stopping(val_score):
            print(f'Epoch [{k}-{epoch}], early stopping')
            break
    if lr_list:
        return (train_loss_list, val_loss_list, best_score, lr_list)
    else:
        return (train_loss_list, val_loss_list, best_score, None)

In [24]:
def k_fold(k): # k-fold
    skf = StratifiedKFold(n_splits=k, shuffle=False)
    f1_sum = 0
    
    for k_, (train_index, valid_index) in enumerate(skf.split(df, df['artist'])):
        print(f'{k_}-fold start')
        
        # class_counts = df.loc[train_index, 'artist'].value_counts(sort=False).to_dict()
        # num_samples = sum(class_counts.values())
        # labels = df.loc[train_index, 'artist'].to_list()
        # class_weights = {l:round(num_samples/(class_counts[l]**0.5), 2) for l in class_counts.keys()}
        # weights = [class_weights[labels[i]] for i in range(int(num_samples))]
        # sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))
        
        train_img_paths, train_labels = get_data(df.iloc[train_index])
        val_img_paths, val_labels = get_data(df.iloc[valid_index])
        train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
        train_loader = DataLoader(
            train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, #sampler=sampler,
            worker_init_fn=seed_worker, generator=g, num_workers=0
        )
        
        val_dataset = CustomDataset(val_img_paths, val_labels, validation_transform)
        val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)
        
        model = EfficientNetModel()
        model.eval()
        optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG['LEARNING_RATE'])
        # lr : 5epochs 동안 0.001->CFG['LEARNING_RATE']
        lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['WARMUP']*len(train_loader), T_mult=1, eta_max=0.001, T_up=50, gamma=0.5, T_hold=CFG['LEARNING_RATE']*10)
        _, _, f1_score, _ = train_epoch(k_, model, optimizer, train_loader, val_loader, lr_scheduler, device)
        f1_sum += f1_score
    return f1_sum/k

In [25]:
f1_score = k_fold(CFG['K-FOLD'])
print(f1_score)

0-fold start
**Epoch [0-1], lr : [0.000953], Train Loss : [2.4740], Val Loss : [1.5937], Val F1 Score : [0.3331]
**Epoch [0-2], lr : [0.000779], Train Loss : [1.7501], Val Loss : [1.0950], Val F1 Score : [0.4930]
**Epoch [0-3], lr : [0.000533], Train Loss : [1.4313], Val Loss : [0.8693], Val F1 Score : [0.5714]
**Epoch [0-4], lr : [0.000271], Train Loss : [1.1688], Val Loss : [0.6619], Val F1 Score : [0.6673]
**Epoch [0-5], lr : [0.000081], Train Loss : [0.9664], Val Loss : [0.6299], Val F1 Score : [0.6948]
Epoch [0-6], lr : [0.000006], Train Loss : [0.8606], Val Loss : [0.6037], Val F1 Score : [0.6902]
EarlyStopping counter: 1 out of 12
Best F1 score from now: 0.6948340641599579
Epoch [0-7], lr : [0.000476], Train Loss : [1.0499], Val Loss : [0.8382], Val F1 Score : [0.5952]
EarlyStopping counter: 2 out of 12
Best F1 score from now: 0.6948340641599579
Epoch [0-8], lr : [0.000392], Train Loss : [1.0309], Val Loss : [0.6928], Val F1 Score : [0.6750]
EarlyStopping counter: 3 out of 12
Be

In [26]:
test_df = pd.read_csv(os.path.join(data_path, './test.csv'))
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [27]:
test_img_paths = get_data(test_df, infer=True)

In [28]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for idx, img in enumerate(test_loader):
            img = img.float().to(device)
            
            model_pred = model(img).detach().cpu()
            model_pred = F.softmax(model_pred, dim=1)
            model_preds.extend(model_pred.numpy().tolist())
    
    print('Done.')
    return model_preds

In [30]:
result_df = pd.DataFrame(np.zeros((test_df.shape[0], len(le.classes_))))
for k_ in range(CFG['K-FOLD']):
    checkpoint = os.path.join(data_path, f'runs/{run_id}/best_model_{k_}.pt')
    print(f'{k_}-fold CHECKPOINT LOADED: {checkpoint}')
    infer_model = torch.load(checkpoint)
    ret = inference(infer_model, test_loader, device)
    result_df += pd.DataFrame(ret)

0-fold CHECKPOINT LOADED: ../../data/runs/20231227082319/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/20231227082319/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/20231227082319/best_model_2.pt
Done.
3-fold CHECKPOINT LOADED: ../../data/runs/20231227082319/best_model_3.pt
Done.
4-fold CHECKPOINT LOADED: ../../data/runs/20231227082319/best_model_4.pt
Done.


In [31]:
result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.001397,0.014461,0.006115,0.009963,0.001255,0.003514,0.005037,0.009877,0.000313,0.029192,...,0.046525,0.025572,0.074419,0.011467,0.014114,0.046805,0.297027,0.005912,0.990717,0.002871
1,0.010086,0.010557,4.299855,0.002922,0.007162,0.002028,0.001507,0.004107,0.024284,0.007705,...,0.007607,0.004476,0.006562,0.043186,0.005298,0.005085,0.012469,0.001932,0.037331,0.004215
2,0.007688,0.002418,0.001069,0.000760,0.000698,0.000374,3.518337,0.000591,0.000243,0.080596,...,0.002441,0.076243,0.019272,0.002810,0.076158,0.005674,0.426654,0.000287,0.005591,0.000468
3,4.499340,0.013153,0.005714,0.006060,0.003788,0.003690,0.000345,0.008508,0.002044,0.001798,...,0.100425,0.001411,0.004092,0.014246,0.001728,0.003125,0.004640,0.002358,0.139713,0.004749
4,0.005345,0.103697,0.766226,0.007619,0.003081,0.032124,0.000546,0.042435,0.052104,0.004950,...,0.014405,0.001115,0.004033,0.009710,0.000993,0.003469,0.008745,0.003246,1.337434,0.001438


In [32]:
preds = result_df.idxmax(axis=1)

In [33]:
preds = le.inverse_transform(preds)

In [34]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [35]:
submit['artist'] = preds

In [36]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Edgar Degas


In [37]:
submit.to_csv(os.path.join(data_path, f"./submit_{CFG['FILENAME']}.csv"), index=False)